In [1]:
!pip install pypsa
!pip install neptune-client
!pip install gymnasium

In [2]:
#Install debugger
!pip install ipdb
import ipdb

In [3]:
import pypsa
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import gc
import psutil
import matplotlib.pyplot as plt

import neptune

from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

[neptune] [warning] NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs-legacy.neptune.ai/setup/upgrading/


In [4]:
# Upload files to Google Drive first, then:
from google.colab import drive
drive.mount('/content/drive')

# Access your files
%cd /content/drive/MyDrive/Colab_Notebooks/networks

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks/networks


In [ ]:
import os
import pickle

def load_mappings(network_file, input_dir="var_constraint_map"):
    """
    Load previously saved variable ID to name mapping and constraints from files in Google Drive.
    
    Parameters:
    -----------
    network_file : str
        Path to the network file used to create the mappings
    input_dir : str, optional
        Directory relative to Google Drive MyDrive where the mapping files are stored
        
    Returns:
    --------
    tuple : (var_id_to_name, constraints) - The loaded mappings
    """
    # Mount Google Drive
    drive.mount('/content/drive')
    
    # Create the full path to Google Drive directory
    gdrive_base = '/content/drive/MyDrive/Colab_Notebooks'
    full_input_dir = os.path.join(gdrive_base, input_dir)
    
    # Get the network filename without path or extension
    network_name = os.path.basename(network_file)
    network_name = os.path.splitext(network_name)[0]
    
    # Create input filenames
    var_map_file = os.path.join(full_input_dir, f"{network_name}_var_id_to_name.pkl")
    constraints_file = os.path.join(full_input_dir, f"{network_name}_constraints.pkl")
    
    # Check if files exist
    if not os.path.exists(var_map_file) or not os.path.exists(constraints_file):
        raise FileNotFoundError(f"Mapping files for {network_name} not found in {full_input_dir}")
    
    # Load mappings from files
    with open(var_map_file, 'rb') as f:
        var_id_to_name = pickle.load(f)
        
    with open(constraints_file, 'rb') as f:
        constraints = pickle.load(f)
    
    print(f"Loaded variable mapping from: {var_map_file}")
    print(f"Loaded constraints from: {constraints_file}")
    
    return var_id_to_name, constraints

In [5]:
def create_pypsa_network(network_file):
    """Create a PyPSA network from the .nc file."""
    # Initialize network
    n = pypsa.Network(network_file)

    return n

In [6]:
def calculate_offset_k_initialization(envClass,k_method='mean', k_samples=1000):
   """
   Calculate the offset k for replacement reward method.
   This ensures valid states always have higher rewards than invalid states.

   Parameters:
   -----------
   k_samples : int
       Number of random samples to use for estimation
   k_method : str
       Method to calculate k: 'mean' or 'worst_case'

   Returns:
   --------
   float: Offset value k
   """
   print(f"Sampling {k_samples} random states to calculate offset k...")

   temp_env = envClass()
   #this initializes episode_length to number of snapshots and constraint_penalty_factor to None.
   #I'm just making this env to access certain attributes/ methods; which should be fine since none of these attributes/methods reference these two parameters.

   objective_values = []
   successful_samples = 0

   try:
       for i in range(k_samples):
           try:
               # STEP 1: Sample ONE random snapshot (random state)
               random_snapshot_idx = np.random.randint(0, temp_env.total_snapshots)

               # STEP 2: Sample ONE random action for that state
               random_action = np.random.random(temp_env.n_non_slack)
               scaled_action = temp_env.scale_action(random_action)

               # STEP 3: Apply the action to the sampled state
               for j, gen_name in enumerate(temp_env.non_slack_names):
                   temp_env.network.generators_t.p_set.iloc[random_snapshot_idx,
                       temp_env.network.generators_t.p_set.columns.get_loc(gen_name)] = scaled_action[j]

               # STEP 4: Evaluate objective for this ONE state-action combination
               temp_env.snapshot_idx = random_snapshot_idx  # Set snapshot for evaluation
               temp_env.network.lpf()
               obj_value = temp_env._evaluate_stored_objective()
               objective_values.append(obj_value)
               successful_samples += 1

               # Progress indicator every 200 samples
               if (i + 1) % 200 == 0:
                   print(f"  Completed {i + 1}/{k_samples} samples...")

           except Exception as e:
               # Skip failed samples but continue
               if i < 5:  # Only print first few errors to avoid spam
                   print(f"  Sample {i} failed: {e}")
               continue

       # Calculate offset based on method
       if objective_values:
           if k_method == 'worst_case':
               k = abs(max(objective_values))
               print(f"  Using worst-case method: k = |{max(objective_values):.2f}| = {k:.2f}")
           else:  # method == 'mean'
               mean_val = np.mean(objective_values)
               k = abs(mean_val)
               print(f"  Using mean method: k = |{mean_val:.2f}| = {k:.2f}")

           print(f"  Successfully sampled {successful_samples}/{k_samples} states")
           print(f"  Objective value range: [{min(objective_values):.2f}, {max(objective_values):.2f}]")
       else:
           print("  Warning: No successful samples, using default k value")
           k = 2500  # Default fallback value

   finally:
       # Clean up temporary environment (optional but good practice)
       del temp_env

   return k

In [ ]:
def get_variable_value(network,var_name):
        """
        Get the current value of a single optimization variable from the network.

        Parameters:
        -----------
        network : pypsa.Network
            The PyPSA network object
        var_name : str
            Variable name like 'Generator-p[snapshot=now,Generator=coal_gen_1]'

        Returns:
        --------
        float : current value of the variable
        """
        # Parse the variable name
        # Format: ComponentName-attribute[dimension1=value1,dimension2=value2,...]

        # Split on first '[' to separate base name from coordinates
        if '[' in var_name:
            base_name, coords_str = var_name.split('[', 1)
            coords_str = coords_str.rstrip(']')  # Remove trailing ']'
        else:
            base_name = var_name
            coords_str = ""

        # Parse base name: ComponentName-attribute
        component_name, attribute = base_name.split('-', 1)

        # Parse coordinates if they exist
        coords = {}
        if coords_str:
            # Split by comma and parse key=value pairs
            for coord_pair in coords_str.split(','):
                key, value = coord_pair.split('=', 1)
                coords[key.strip()] = value.strip()

        # Determine if this has time dimension (snapshot)
        has_snapshot = 'snapshot' in coords

        if has_snapshot:
            # Access dynamic dataframe using n.dynamic()
            snapshot_value = coords['snapshot']
            component_instance = coords[component_name]

            # Special handling for branch flow variables
            if component_name in network.passive_branch_components and attribute == 's':
                # For branch components, 's' is stored as 'p0' in the network
                # We can use p0 directly as the value of 's'
                try:
                    return network.dynamic(component_name)['p0'].loc[snapshot_value, component_instance]
                except Exception as e:
                    print(f"Warning: Could not get flow value for {var_name}: {e}")
                    return 0.0

            # Get dynamic data for normal case
            dynamic_data = network.dynamic(component_name)

            # Access the specific attribute DataFrame
            if attribute in dynamic_data:
                return dynamic_data[attribute].loc[snapshot_value, component_instance]
            else:
                raise KeyError(f"Attribute {attribute} not found in dynamic data for {component_name}")

        else:
            # Access static dataframe using n.static()
            component_instance = coords[component_name]

            # Get static data
            static_data = network.static(component_name)

            # Access the value
            return static_data.loc[component_instance, attribute]

def create_variable_values_mapping(network, variable_names):
        """
        Create a mapping from optimization variable names to their current values in the network.

        Parameters:
        -----------
        network : pypsa.Network
            The PyPSA network object
        variable_names : list
            List of variable names like ['Generator-p[snapshot=now,Generator=coal_gen_1]', ...]

        Returns:
        --------
        dict : mapping from variable name to current value
        """
        var_values = {}

        for var_name in variable_names:
            try:
                value = get_variable_value(network,var_name)
                var_values[var_name] = value
            except Exception as e:
                print(f"Warning: Could not get value for {var_name}: {e}")
                var_values[var_name] = 0.0  # Default fallback

        return var_values

def evaluate_objective(var_id_to_name, network, snapshot_idx, only_generator_terms= True):
        """
        Direct evaluation without mock objects.
        Only includes terms from the current snapshot in the objective function.
        """
        temp_model = network.optimize.create_model()

        # Extract objective components
        obj_expr = temp_model.objective
        objective_vars = obj_expr.vars.copy()
        objective_coeffs = obj_expr.coeffs.copy()
        objective_const = obj_expr.const.copy() if hasattr(obj_expr, 'const') else 0

        # Get variable name mapping for current network
        id_to_name = var_id_to_name

        # Get the current snapshot name
        current_snapshot = snapshot_idx

        # Get current variable values
        variable_names = []
        var_indices = []
        vars_flat = objective_vars.values.flatten()
        coeffs_flat = objective_coeffs.values.flatten()

        # Filter variables to only include those from the current snapshot
        for i, var_id in enumerate(vars_flat):
            if var_id != -1 and var_id in id_to_name:
                var_name = id_to_name[var_id]

                # Filter for generator-related terms if requested
                if only_generator_terms:
                    # Check if this is a generator-related variable
                    # Adjust these patterns based on your specific variable naming
                    is_generator_var = ('generator' in var_name.lower())

                    if not is_generator_var:
                        continue  # Skip non-generator variables

                # Check if this variable belongs to the current snapshot
                if 'snapshot=' in var_name:
                    # Extract the snapshot value from the variable name
                    snapshot_part = var_name.split('snapshot=')[1].split(',')[0].split(']')[0]
                    if snapshot_part == str(current_snapshot):
                        variable_names.append(var_name)
                        var_indices.append(i)
                else:
                    # Include variables without snapshot dimension (like investment variables)
                    variable_names.append(var_name)
                    var_indices.append(i)

        var_values = create_variable_values_mapping(network,variable_names)

        # Direct mathematical evaluation using only variables from current snapshot
        if var_indices:
            result = np.sum(coeffs_flat[var_indices] *
                        [var_values.get(name, 0) for name in variable_names]) + \
                    objective_const
        else:
            # If no variables for this snapshot, just return the constant
            result = objective_const

        return result

In [ ]:
def evaluate_baseline_reward(network_file, env, var_id_to_name):
    network_baseline=create_pypsa_network(network_file)#network to run pypsa optimize on
    network_baseline.optimize()
    snapshots=network_baseline.snapshots
    objective_sum=0
    for snapshot_idx in range(len(snapshots)):
        objective_sum+=evaluate_objective(var_id_to_name, network_baseline, snapshots[snapshot_idx])
    baseline_reward_value=-objective_sum*(env.episode_length/ len(snapshots))#assuming episode length is a multiple of the number of snapshots
    #reward is -1 times the objective value
    return baseline_reward_value

In [ ]:
class Env2Gen1LoadConstr(gym.Env):
    """
    OpenAI Gym environment for Optimal Power Flow using PyPSA.
    Simple example with 2 generators and 1 load.
    Implemented without constraints (omitted bus voltage limits, line limits, etc.).

    Action Space: Continuous setpoints for generators within their capacity limits
    Observation Space: Network state, represented by:
    - the active and reactive power of all loads,
    - the reactive power prices of all generators,
    - the active power setpoints of all generators
    (This follows http://arxiv.org/abs/2403.17831. Additional variables can be added optionally)
    """

    def __init__(self,network_file, episode_length=None, constraint_penalty_factor=100, input_dir="var_constraint_map"):
        super(Env2Gen1LoadConstr, self).__init__()

        # Use provided network or create new one
        self.network =create_pypsa_network(network_file)
        self.network.optimize() #only do this if you want RL agent to control dispatch of generators
        #It will call pypsa's optimization to set all other controllable component attributes to optimal value.
        #Later we reset the generator dynamic attributes so that the agent can choose these

        #this is if you want RL agent only to control dispatch of generators
        self._initialize_optimization_components()
        self.penalty_factor=constraint_penalty_factor

        # Episode management
        self.total_snapshots = len(self.network.snapshots)
        self.episode_length = episode_length if episode_length is not None else self.total_snapshots
        self.current_step = 0  # Steps within current episode
        self.snapshot_idx = 0  # Current snapshot index (cycles through all snapshots)

        # Find the slack generator
        # The agent will only be able to control the active power setpoints of generators that are not the slack generator
        self.slack_generator_idx = self.network.generators[self.network.generators.control == "Slack"].index
        #Note that there should only ever be one slack bus.
        #Need to implement handling for multiple generators control set to "Slack",
        #In that case, will need to designate one for slack and change control of others
        #TO DO: Implement handling for multiple slack generators

        #Get indices of all generators that are not the slack generator (after ensuring there is only one slack generator)
        self.non_slack_gens = self.network.generators[self.network.generators.control != "Slack"].index #must explicitly search again, can't reference results of last query since this may have changed since choosing slack generator
        self.non_slack_names = list(self.non_slack_gens)
        self.n_non_slack = len(self.non_slack_names)
        self.non_slack_df = self.network.generators.loc[self.non_slack_gens]  # Fixed: was non_slack_generators
        # The agent will only be able to control the active power setpoints of generators that are not the slack generator

        # Get generator limits (in MW)
        self.a_min = (self.non_slack_df.p_min_pu * self.non_slack_df.p_nom).values
        self.a_max = (self.non_slack_df.p_max_pu * self.non_slack_df.p_nom).values
        #TO DO: When generalize, control of the generator will determine the action space of the agent
        # Whichever quantity is controllable, according to the control, will be the quantity the agent chooses as its action (e.g. could be active or reactive power)
        # Then we would need to change the bounds of the the relevant bounds will be the actions space to be the min and max of whatever the quantity is

        # Define action space: continuous setpoints for each generator within limits
        # Action space is a Box with shape (n_non_slack,) where each element has value between 0 and 1.
        self.action_space = gym.spaces.Box(0, 1, shape=(self.n_non_slack,))

        # Define observation space - network state
        # This will include: generator outputs, loads, voltages, line flows, etc.
        # For simplicity, let's include key network variables
        # obs_dim = (self.n_non_slack+1)+len(self.network.loads)
        # First term: self.n_non_slack: Active power outputs of all non-slackgenerators! Actually I removed this because that actually corresponds to the action choice.
        # TO DO: Later we might need to include a history of the network as part of the observation - to obey ramp constraints etc.
        #Second term: Cost function components (start with one each - marginal cost) of all generators, including slack
        # Third term: Active power of load demands
        # TO DO: include power of storage units and noncontrollable generators (small generators and storage units; not including slack generator) in observation space


        #TO DO: If use .pf instead of .lpf, add another of each term for active power AND reactive power
        #include the power of the slack generator in the observation, since agent's action (choice of active power for non-slack generators influences power of slack generator, through power flow calculations)

        # Initialize the network state
        self.reset()

        # Observation space is bounded - you may want to adjust these bounds based on your system

        # Create observation space
        low_bounds, high_bounds = self.create_observation_bounds()
        self.observation_space = spaces.Box(
            low=low_bounds,
            high=high_bounds,
            dtype=np.float32
        )

        #Bounds are set according to specific component of the observation
        #Could set the bounds of active power of each generator to correspond to min and max p

    def _initialize_optimization_components(self):
        """
        Initialize all optimization components in one pass to avoid creating multiple models.
        This method:
        1. Creates the optimization model once
        2. Extracts objective components (vars, coeffs, const)
        3. Creates the variable ID to name mapping
        4. Extracts constraints
        5. Cleans up the model
        """
        # Create model once - this is an expensive operation
        temp_model = self.network.optimize.create_model()

        # Extract objective components
        obj_expr = temp_model.objective
        self.objective_vars = obj_expr.vars.copy()
        self.objective_coeffs = obj_expr.coeffs.copy()
        self.objective_const = obj_expr.const.copy() if hasattr(obj_expr, 'const') else 0

        self.var_id_to_name, self.constraints = load_mappings(self.network_file,self.input_dir="var_constraint_map")

        # Clean up to free memory
        del temp_model, obj_expr
        gc.collect()

    def create_observation_bounds(self):
        """
        Create bounds for the observation space based on generator costs and load power
        """
        # --- Generator Cost Bounds ---
        gen_cost_lower = []
        gen_cost_upper = []

        for gen in self.network.generators.index:
            # Get generator specs
            p_nom = self.network.generators.loc[gen, "p_nom"]
            p_min_pu = self.network.generators.loc[gen, "p_min_pu"]
            p_max_pu = self.network.generators.loc[gen, "p_max_pu"]
            marginal_cost = self.network.generators.loc[gen, "marginal_cost"]

            # Min cost: generator at minimum output
            min_cost = p_min_pu * p_nom * marginal_cost

            # Max cost: generator at maximum output
            max_cost = p_max_pu * p_nom * marginal_cost
            #TO DO: add other cost function components if applicable
            gen_cost_lower.append(min_cost)
            gen_cost_upper.append(max_cost)

        # --- Load Power Bounds ---
        load_p_lower = []
        load_p_upper = []

        for load in self.network.loads.index:
            # Get load specs
            p_nom = self.network.loads.loc[load, "p_set"]#TO DO: not sure if this is correct. in general might not set p_set when specifying load component (might instead specifiy p_nom)

            # If you have time series data for loads
            if hasattr(self.network, "loads_t") and not self.network.loads_t.p.empty:
                min_load = self.network.loads_t.p.iloc[self.snapshot_idx].min()
                max_load = self.network.loads_t.p.iloc[self.snapshot_idx].max()

            load_p_lower.append(min_load)
            load_p_upper.append(max_load)

        # Combine all bounds
        low_bounds = np.concatenate([gen_cost_lower, load_p_lower]).astype(np.float32)
        high_bounds = np.concatenate([gen_cost_upper, load_p_upper]).astype(np.float32)

        return low_bounds, high_bounds

    def reset_network(self):
        """Reset and ensure essential DataFrames exist."""
        #Note that we do not just create a new network here, as this consumes more memory and previously led to a segmentation fault
        # TO DO: For a general network, we would need to reset all time-varying data (i.e. all components with time-varying data)
        # Ensure generators_t DataFrames exist and are reset
        if not hasattr(self.network, 'generators_t'):
            # This shouldn't happen with PyPSA, but just in case
            pass

        # Initialize/reset generators_t.p_set
        if not hasattr(self.network.generators_t, 'p_set') or self.network.generators_t.p_set.empty:
            self.network.generators_t.p_set = pd.DataFrame(
                0.0,
                index=self.network.snapshots,
                columns=self.network.generators.index
            )
        else:
            self.network.generators_t.p_set.iloc[:, :] = 0.0

    def reset(self, seed=None, options=None):
        """
        Reset the environment to initial state.
        Returns initial observation and info (gymnasium format).
        """
        # Set seed if provided
        if seed is not None:
            self.seed(seed)

        # Reset episode counters
        self.current_step = 0
        self.snapshot_idx = 0

        self.reset_network()
        #self._test_hidden_references()
        #can run that instead to check if there are hidden references preventing network cleanup

        # Get initial observation
        obs = self._get_observation()

        # Info dict for gymnasium compatibility
        info = {
            'current_step': self.current_step,
            'snapshot_idx': self.snapshot_idx
        }

        return obs, info

    def _get_observation(self):
        """
        Get current network state as observation.
        """
        obs_components = []

        # obs_dim = (self.n_non_slack+1)+len(self.network.loads)
        # First term: self.n_non_slack: Active power outputs of all non-slackgenerators! Actually I removed this because that actually corresponds to the action choice.
        # TO DO: Later we might need to include a history of the network as part of the observation - to obey ramp constraints etc.
        # Second term: Cost function components (start with one each - marginal cost) of all generators, including slack
        # Third term: Active power of load demands


        # Generator power costs (for now just marginal) - TO DO: change this to include more components of cost function to make this more accurate
        gen_costs = self.network.generators.marginal_cost.values #TO DO: not sure if the marginal_cost should be accessed from generators_t instead (would do generators.marginal_cost.iloc[snapshot_idx].values). But then not static
        obs_components.append(gen_costs)

        load_demands = np.array([])
        # Load demands - handle case where we might have empty loads_t.p
        if hasattr(self.network, 'loads_t') and not self.network.loads_t.p.empty:
            load_demands = self.network.loads_t.p.iloc[self.snapshot_idx].values
        obs_components.append(load_demands)

        #The following line should be uncommented when want to enforce network constraints via action masking
        #obs_components.append(self._get_mask())

        # TO DO: Could later try to add bus voltages etc. and see if difference on agent learning. (Paper says these are optional components of the observation space)
        # Concatenate all observation components
        observation = np.concatenate(obs_components).astype(np.float32)

        return observation

    def scale_action(self, action):
        """
        Scale action from [0,1] range to [self.a_min, self.a_max] range.

        Parameters:
        -----------
        action : numpy.ndarray
            Action values in range [0,1]

        Returns:
        --------
        numpy.ndarray
            Scaled action in range [self.a_min, self.a_max]
        """
        return self.a_min + action * (self.a_max - self.a_min)

    def get_variable_value(self, var_name):
        """
        Get the current value of a single optimization variable from the network.

        Parameters:
        -----------
        network : pypsa.Network
            The PyPSA network object
        var_name : str
            Variable name like 'Generator-p[snapshot=now,Generator=coal_gen_1]'

        Returns:
        --------
        float : current value of the variable
        """
        # Parse the variable name
        # Format: ComponentName-attribute[dimension1=value1,dimension2=value2,...]

        # Split on first '[' to separate base name from coordinates
        if '[' in var_name:
            base_name, coords_str = var_name.split('[', 1)
            coords_str = coords_str.rstrip(']')  # Remove trailing ']'
        else:
            base_name = var_name
            coords_str = ""

        # Parse base name: ComponentName-attribute
        component_name, attribute = base_name.split('-', 1)

        # Parse coordinates if they exist
        coords = {}
        if coords_str:
            # Split by comma and parse key=value pairs
            for coord_pair in coords_str.split(','):
                key, value = coord_pair.split('=', 1)
                coords[key.strip()] = value.strip()

        # Determine if this has time dimension (snapshot)
        has_snapshot = 'snapshot' in coords

        if has_snapshot:
            # Access dynamic dataframe using n.dynamic()
            snapshot_value = coords['snapshot']
            component_instance = coords[component_name]

            # Special handling for branch flow variables
            if component_name in self.network.passive_branch_components and attribute == 's':
                # For branch components, 's' is stored as 'p0' in the network
                # We can use p0 directly as the value of 's'
                try:
                    return self.network.dynamic(component_name)['p0'].loc[snapshot_value, component_instance]
                except Exception as e:
                    print(f"Warning: Could not get flow value for {var_name}: {e}")
                    return 0.0

            # Get dynamic data for normal case
            dynamic_data = self.network.dynamic(component_name)

            # Access the specific attribute DataFrame
            if attribute in dynamic_data:
                return dynamic_data[attribute].loc[snapshot_value, component_instance]
            else:
                raise KeyError(f"Attribute {attribute} not found in dynamic data for {component_name}")

        else:
            # Access static dataframe using n.static()
            component_instance = coords[component_name]

            # Get static data
            static_data = self.network.static(component_name)

            # Access the value
            return static_data.loc[component_instance, attribute]


    def create_variable_values_mapping(self,variable_names):
        """
        Create a mapping from optimization variable names to their current values in the network.

        Parameters:
        -----------
        network : pypsa.Network
            The PyPSA network object
        variable_names : list
            List of variable names like ['Generator-p[snapshot=now,Generator=coal_gen_1]', ...]

        Returns:
        --------
        dict : mapping from variable name to current value
        """
        var_values = {}

        for var_name in variable_names:
            try:
                value = self.get_variable_value(var_name)
                var_values[var_name] = value
            except Exception as e:
                print(f"Warning: Could not get value for {var_name}: {e}")
                var_values[var_name] = 0.0  # Default fallback

        return var_values

    def _evaluate_stored_objective(self,  only_generator_terms=True):
        """
        Direct evaluation without mock objects.
        Only includes terms from the current snapshot in the objective function.
        """
        # Get variable name mapping for current network
        id_to_name = self.var_id_to_name

        # Get the current snapshot name
        current_snapshot = self.network.snapshots[self.snapshot_idx]

        # Get current variable values
        variable_names = []
        var_indices = []
        vars_flat = self.objective_vars.values.flatten()
        coeffs_flat = self.objective_coeffs.values.flatten()

        # Filter variables to only include those from the current snapshot
        for i, var_id in enumerate(vars_flat):
            if var_id != -1 and var_id in id_to_name:
                var_name = id_to_name[var_id]

                # Filter for generator-related terms if requested
                if only_generator_terms:
                    # Check if this is a generator-related variable
                    # Adjust these patterns based on your specific variable naming
                    is_generator_var = ('generator' in var_name.lower())
                    if not is_generator_var:
                        continue  # Skip non-generator variables


                # Check if this variable belongs to the current snapshot
                if 'snapshot=' in var_name:
                    # Extract the snapshot value from the variable name
                    snapshot_part = var_name.split('snapshot=')[1].split(',')[0].split(']')[0]
                    if snapshot_part == str(current_snapshot):
                        variable_names.append(var_name)
                        var_indices.append(i)
                else:
                    # Include variables without snapshot dimension (like investment variables)
                    variable_names.append(var_name)
                    var_indices.append(i)

        var_values = self.create_variable_values_mapping(variable_names)

        # Direct mathematical evaluation using only variables from current snapshot
        if var_indices:
            result = np.sum(coeffs_flat[var_indices] *
                        [var_values.get(name, 0) for name in variable_names]) + \
                    self.objective_const
        else:
            # If no variables for this snapshot, just return the constant
            result = self.objective_const

        return result

    def _evaluate_constraint(self, constraint_key):
        """
        Evaluate a single constraint using current network values.

        Parameters:
        -----------
        constraint_key : str
            Key of the specific constraint to evaluate

        Returns:
        --------
        tuple: (bool, float)
            Boolean indicating if constraint is satisfied, and violation amount (0 if satisfied)
        """
        try:
            if constraint_key not in self.constraints:
                return True, 0

            constraint = self.constraints[constraint_key]
            lhs = constraint['lhs']
            rhs = constraint['rhs']
            sign = constraint['sign']
        except Exception as e:
            print(f"Error accessing constraint {constraint_key}: {e}")
            return True, 0  # Default to no violation on error

        # Get variable name mapping for current network
        id_to_name = self.var_id_to_name

        # Evaluate LHS if it's a linear expression
        lhs_value = 0
        if hasattr(lhs, 'vars') and hasattr(lhs, 'coeffs'):
            # Get current variable values
            variable_names = []
            vars_flat = lhs.vars.flatten()  # Flatten to handle any dimensionality
            coeffs_flat = lhs.coeffs.flatten()  # Flatten coefficients as well

            # Create a mask for valid variable indices (not -1)
            valid_indices = vars_flat != -1

            # Filter out -1 indices
            valid_vars = vars_flat[valid_indices]
            valid_coeffs = coeffs_flat[valid_indices]

            # Get variable names only for valid indices
            for var_id in valid_vars:
                if var_id in id_to_name:
                    variable_names.append(id_to_name[var_id])

            # Get variable values
            var_values = self.create_variable_values_mapping(variable_names)

            # Direct mathematical evaluation using only valid variables and coefficients
            try:
                # Convert variable values to numpy array
                var_values_array = np.array([var_values.get(name, 0) for name in variable_names])

                # Calculate LHS value using only valid variables and coefficients
                lhs_value = np.sum(valid_coeffs * var_values_array)

                # Add constant if it exists
                if hasattr(lhs, 'const'):
                    lhs_value += lhs.const.item() if hasattr(lhs.const, 'item') else lhs.const

            except Exception as e:
                print(f"Error in constraint evaluation for {constraint_key}: {e}")
                return True, 0  # Skip this constraint on error
        else:
            # If it's a constant or simple value
            lhs_value = lhs

        # Get RHS value (will be a constant)
        rhs_value = rhs

        # Check constraint satisfaction based on sign
        try:
            # Handle both scalar and array values
            if sign == '==':
                if isinstance(lhs_value, (np.ndarray, list)) or isinstance(rhs_value, (np.ndarray, list)):
                    # Convert to numpy arrays if needed
                    lhs_array = np.asarray(lhs_value)
                    rhs_array = np.asarray(rhs_value)

                    # Handle shape mismatches
                    if lhs_array.shape != rhs_array.shape:
                        if np.isscalar(lhs_array) or len(lhs_array.shape) == 0:
                            lhs_array = np.full_like(rhs_array, lhs_array)
                        elif np.isscalar(rhs_array) or len(rhs_array.shape) == 0:
                            rhs_array = np.full_like(lhs_array, rhs_array)
                        else:
                            # If shapes still don't match, return no violation
                            print(f"Shape mismatch in constraint {constraint_key}: {lhs_array.shape} vs {rhs_array.shape}")
                            return True, 0

                    satisfied = np.all(np.isclose(lhs_array, rhs_array))
                    violation = float(np.sum(np.abs(lhs_array - rhs_array)))
                else:
                    satisfied = np.isclose(lhs_value, rhs_value)
                    violation = float(abs(lhs_value - rhs_value))
            elif sign == '<=':
                if isinstance(lhs_value, (np.ndarray, list)) or isinstance(rhs_value, (np.ndarray, list)):
                    # Convert to numpy arrays if needed
                    lhs_array = np.asarray(lhs_value)
                    rhs_array = np.asarray(rhs_value)

                    # Handle shape mismatches
                    if lhs_array.shape != rhs_array.shape:
                        if np.isscalar(lhs_array) or len(lhs_array.shape) == 0:
                            lhs_array = np.full_like(rhs_array, lhs_array)
                        elif np.isscalar(rhs_array) or len(rhs_array.shape) == 0:
                            rhs_array = np.full_like(lhs_array, rhs_array)
                        else:
                            # If shapes still don't match, return no violation
                            print(f"Shape mismatch in constraint {constraint_key}: {lhs_array.shape} vs {rhs_array.shape}")
                            return True, 0

                    satisfied = np.all(lhs_array <= rhs_array)
                    violation = float(np.sum(np.maximum(0, lhs_array - rhs_array)))
                else:
                    satisfied = lhs_value <= rhs_value
                    violation = float(max(0, lhs_value - rhs_value))
            elif sign == '>=':
                if isinstance(lhs_value, (np.ndarray, list)) or isinstance(rhs_value, (np.ndarray, list)):
                    # Convert to numpy arrays if needed
                    lhs_array = np.asarray(lhs_value)
                    rhs_array = np.asarray(rhs_value)

                    # Handle shape mismatches - don't think I need this
                    if lhs_array.shape != rhs_array.shape:
                        if np.isscalar(lhs_array) or len(lhs_array.shape) == 0:
                            lhs_array = np.full_like(rhs_array, lhs_array)
                        elif np.isscalar(rhs_array) or len(rhs_array.shape) == 0:
                            rhs_array = np.full_like(lhs_array, rhs_array)
                        else:
                            # If shapes still don't match, return no violation
                            print(f"Shape mismatch in constraint {constraint_key}: {lhs_array.shape} vs {rhs_array.shape}")
                            return True, 0

                    satisfied = np.all(lhs_array >= rhs_array)
                    violation = float(np.sum(np.maximum(0, rhs_array - lhs_array)))
                else:
                    satisfied = lhs_value >= rhs_value
                    violation = float(max(0, rhs_value - lhs_value))
            else:
                # Unknown sign
                satisfied = True
                violation = 0

            # Ensure violation is a scalar
            if hasattr(violation, '__len__'):
                violation = float(np.sum(violation))

        except Exception as e:
            print(f"Error comparing constraint values for {constraint_key}: {e}")
            print(f"LHS type: {type(lhs_value)}, RHS type: {type(rhs_value)}")
            print(f"LHS: {lhs_value}, RHS: {rhs_value}")
            satisfied = True
            violation = 0

        return satisfied, violation

    def _evaluate_all_constraints(self):
        """
        Evaluate constraints relevant to the current snapshot.
        Only includes constraints that:
        1. Are for the current snapshot only, or
        2. Link the current snapshot with previous snapshots (but not future snapshots)

        Returns:
        --------
        dict: Information about constraint violations
        """
        results = {
            'all_satisfied': True,
            'violations': {},
            'total_violation': 0.0,
            'violations_by_group': {}  # Track violations by constraint group
        }

        try:
            # Get the current snapshot name
            current_snapshot = self.network.snapshots[self.snapshot_idx]

            # Evaluate each individual constraint that's relevant to the current snapshot
            for constraint_key in self.constraints:
                try:
                    # Check if this constraint is relevant to the current snapshot
                    is_relevant = False

                    # If constraint has no snapshot specification, include it
                    if 'snapshot=' not in constraint_key:
                        is_relevant = True
                    else:
                        # Extract all snapshots mentioned in this constraint
                        constraint_snapshots = []
                        parts = constraint_key.split('snapshot=')
                        for i in range(1, len(parts)):
                            snapshot_val = parts[i].split(',')[0].split(']')[0]
                            constraint_snapshots.append(snapshot_val)

                        # Include if current snapshot is mentioned
                        if str(current_snapshot) in constraint_snapshots:
                            # Check if any future snapshots are mentioned
                            has_future_snapshots = False
                            for snap in constraint_snapshots:
                                try:
                                    # Find the index of this snapshot in the network's snapshots
                                    snap_idx = list(self.network.snapshots).index(snap)
                                    if snap_idx > self.snapshot_idx:
                                        has_future_snapshots = True
                                        break
                                except (ValueError, TypeError):
                                    # If snapshot can't be found or compared, skip this check
                                    pass

                            # Include if no future snapshots are mentioned
                            if not has_future_snapshots:
                                is_relevant = True

                    # Only evaluate if the constraint is relevant
                    if is_relevant:
                        satisfied, violation = self._evaluate_constraint(constraint_key)
                        if not satisfied:
                            results['all_satisfied'] = False
                            results['violations'][constraint_key] = float(violation)  # Ensure it's a scalar
                            results['total_violation'] += float(violation)

                            # Also track violations by constraint group (original name without coordinates)
                            if '[' in constraint_key:
                                group_name = constraint_key.split('[')[0]
                                if group_name not in results['violations_by_group']:
                                    results['violations_by_group'][group_name] = 0.0
                                results['violations_by_group'][group_name] += float(violation)
                except Exception as e:
                    print(f"Error evaluating constraint {constraint_key}: {e}")
                    # Continue with other constraints
        except Exception as e:
            print(f"Error in constraint evaluation: {e}")
            # Return default results

        return results

    def calculate_constrained_reward(self):
        """
        Calculate reward with constraint violation penalties.

        Returns:
        --------
        float: Reward value with constraint penalties
        """
        try:
            # Get base reward from objective function
            base_reward = self._calculate_reward()

            # Evaluate constraints
            constraint_results = self._evaluate_all_constraints()

            # Apply penalty for constraint violations
            # Using a high penalty factor to make violations very noticeable
            # Increased from 100.0 to make violations more obvious

            # Ensure total_violation is a scalar
            total_violation = float(constraint_results['total_violation'])
            penalty = self.penalty_factor * total_violation

            # Final reward is base reward minus penalties
            constrained_reward = base_reward - penalty

            # Ensure reward is a scalar
            if hasattr(constrained_reward, '__len__'):
                constrained_reward = float(constrained_reward)

            return constrained_reward, constraint_results
        except Exception as e:
            print(f"Error calculating constrained reward: {e}")
            # Fall back to unconstrained reward
            return self._calculate_reward()

    def _calculate_reward(self):
        """Calculate reward using stored objective components."""
        # Create a minimal mock expression or use your evaluation directly
        return -1 * self._evaluate_stored_objective()

    def step(self, action):
        """
        Execute one time step within the environment.

        Args:
            action: Array of generator setpoints [gen1_setpoint, gen2_setpoint, ...]

        Returns:
            observation: Network state after action
            reward: Reward for this action
            terminated: Whether episode is finished due to task completion
            truncated: Whether episode is finished due to time limit
            info: Additional information
        """
        scaled_action = self.scale_action(action)

        # Update generator setpoints for non-slack generators only
        for i, gen_name in enumerate(self.non_slack_names):
            self.network.generators_t.p_set.iloc[self.snapshot_idx,
                self.network.generators_t.p_set.columns.get_loc(gen_name)] = scaled_action[i]
        # Run power flow to get new network state
        try:
            # You can choose linear or non-linear power flow
            self.network.lpf()  # Linear power flow
            # print(self.network.generators_t.p.loc['now'])  # Commented out to reduce output
            # self.network.pf()  # Non-linear power flow (alternative)

            power_flow_converged = True
        except Exception as e:
            print(f"Power flow failed: {e}")
            power_flow_converged = False

        # Calculate reward using constrained reward function
        reward, constraint_results = self.calculate_constrained_reward()
        #reward=self._calculate_reward()

        # Increment step counters
        self.current_step += 1
        self.snapshot_idx += 1

        # Handle cycling through snapshots
        if self.snapshot_idx >= self.total_snapshots:
            self.snapshot_idx = 0  # Reset to beginning
            self.reset_network()
            #self._test_hidden_references()
            #can run that instead to check if there are hidden references preventing network cleanup


        # Get new observation
        observation = self._get_observation()

        # Check if episode is done
        episode_done = self._check_done()

        # In gymnasium, we need to separate terminated vs truncated
        terminated = False  # Task completion (not applicable here)
        truncated = episode_done  # Time limit reached

        # Additional info
        info = {
            'generator_setpoints': scaled_action,
            'power_flow_converged': power_flow_converged,
            'generator_names': self.non_slack_names,
            'current_step': self.current_step,
            'snapshot_idx': self.snapshot_idx,
            'constraints_satisfied': constraint_results['all_satisfied'],
            'constraint_violations': constraint_results['violations'],
            'total_violation': constraint_results['total_violation']
        }

        return observation, reward, terminated, truncated, info



    def _check_done(self):
        """
        Check if episode should terminate.

        Episode terminates when we've reached the specified episode length.
        """
        if self.current_step >= self.episode_length:
            return True

        # TO DO: add other cases might want to terminate
        # You might want to terminate on:
        # - Power flow convergence failure
        # - Voltage limit violations
        # - Line overloads

        return False

    def seed(self, seed=None):
        """
        Set the random seed for reproducible experiments.
        """
        np.random.seed(seed)
        return [seed]

    def get_constraint_info(self):
        """
        Get detailed information about the constraints in the model.

        Returns:
        --------
        dict: Detailed information about constraints and their current status
        """
        # Evaluate all constraints
        all_results = self._evaluate_all_constraints()

        # Organize constraint information by groups and individual constraints
        constraint_info = {
            'all_satisfied': all_results['all_satisfied'],
            'total_violation': all_results['total_violation'],
            'groups': {},
            'individual': {}
        }

        # Group constraints by their base name
        for constraint_key in self.constraints:
            # Evaluate this specific constraint
            satisfied, violation = self._evaluate_constraint(constraint_key)

            # Store individual constraint info
            constraint_info['individual'][constraint_key] = {
                'satisfied': satisfied,
                'violation': violation,
                'sign': self.constraints[constraint_key]['sign']
            }

            # Also group by constraint type
            if '[' in constraint_key:
                group_name = constraint_key.split('[')[0]
                if group_name not in constraint_info['groups']:
                    constraint_info['groups'][group_name] = {
                        'count': 0,
                        'satisfied_count': 0,
                        'violated_count': 0,
                        'total_violation': 0.0,
                        'max_violation': 0.0,
                        'sign': self.constraints[constraint_key]['sign']
                    }

                group_info = constraint_info['groups'][group_name]
                group_info['count'] += 1

                if satisfied:
                    group_info['satisfied_count'] += 1
                else:
                    group_info['violated_count'] += 1
                    group_info['total_violation'] += violation
                    group_info['max_violation'] = max(group_info['max_violation'], violation)
            else:
                # Handle standalone constraints
                if 'standalone' not in constraint_info['groups']:
                    constraint_info['groups']['standalone'] = {
                        'count': 0,
                        'satisfied_count': 0,
                        'violated_count': 0,
                        'total_violation': 0.0,
                        'max_violation': 0.0
                    }

                group_info = constraint_info['groups']['standalone']
                group_info['count'] += 1

                if satisfied:
                    group_info['satisfied_count'] += 1
                else:
                    group_info['violated_count'] += 1
                    group_info['total_violation'] += violation
                    group_info['max_violation'] = max(group_info['max_violation'], violation)

        return constraint_info

    def render(self, mode='human', info=None):
        """
        Render the environment state.

        Parameters:
        -----------
        mode : str
            Rendering mode (only 'human' supported)
        info : dict, optional
            Information dictionary from step() method containing constraint data
        """
        print("=== Current Network State ===")
        print(f"Episode step: {self.current_step}/{self.episode_length}")
        print(f"Snapshot index: {self.snapshot_idx}/{self.total_snapshots}")
        print(f"Current snapshot: {self.network.snapshots[self.snapshot_idx]}")
        print(f"Generator setpoints: {self.network.generators_t.p_set.iloc[self.snapshot_idx].values}")
        print(f"Load values: {self.network.loads_t.p_set.iloc[self.snapshot_idx].values}")

        all_satisfied = info['constraints_satisfied']
        total_violation = info['total_violation']
        violations = info['constraint_violations']


        print(f"All constraints satisfied: {all_satisfied}")
        print(f"Total constraint violation: {total_violation:.4f}")

        # Show violated constraints if any
        if not all_satisfied and violations:
            print("\n=== Constraint Violations ===")
            for constraint_name, violation in violations.items():
                print(f"  {constraint_name}: {violation:.4f}")

In [21]:
env=Env2Gen1LoadConstr(network_file="elec_s_5_ec_lc1.0_3h.nc")
print(hasattr(env.network, "loads_t") and not env.network.loads_t.p.empty)

Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA1 0', 'ZA2 0', 'ZA3 0'], dtype='object', name='Bus')
Writing continuous variables.: 100%|██████████| 6/6 [00:00<00:00, 20.72it/s]
Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: Infeasible

Index(['0'], dtype='object', name='SubNetwork')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA1 0', 'ZA2 0', 'ZA3 0'], dtype='object', name='Bus')


KeyboardInterrupt: 

In [10]:
class BackboneNetwork(nn.Module):
    def __init__(self, input_features, hidden_dimensions, out_features, dropout):
        super(BackboneNetwork, self).__init__()

        # SIMPLIFIED: Single hidden layer network for debugging
        self.neuralnet = nn.Sequential(
            nn.Linear(input_features, hidden_dimensions),
            nn.ReLU(),
            nn.Linear(hidden_dimensions, hidden_dimensions),
            nn.ReLU(),
            nn.Linear(hidden_dimensions, out_features)
        )

    def forward(self, x):
        output = self.neuralnet(x)
        return output

#Define the actor-critic network
class actorCritic(nn.Module):
    def __init__(self, actor, critic):
        super().__init__()
        self.actor = actor
        self.critic = critic
    def forward(self, state):
        action_pred = self.actor(state)
        value_pred = self.critic(state)
        return action_pred, value_pred
        #Returns both the action predictions and the value predictions.

#We'll use the networks defined above to create an actor and a critic. Then, we will create an agent, including the actor and the critic.
#finish this step later
# def create_agent(hidden_dimensions, dropout):
#     INPUT_FEATURES =env_train.
class PPO_agent:
    def __init__(self,
                 env,
                 device,
                 run,
                 hidden_dimensions,
                 dropout, discount_factor,
                 max_episodes,
                 print_interval,
                 PPO_steps,
                 n_trials,
                 epsilon,
                 entropy_coefficient,
                 learning_rate,
                 batch_size,
                 optimizer_name,
                 seed):

        self.seed = seed
        if seed is not None:
            # Set PyTorch seed for this class
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed(seed)

        self.env = env  # Store the environment as an attribute

        self.device = device
        self.run = run

        # Get observation and action space dimensions for gymnasium environment
        obs, _ = self.env.reset()

        self.INPUT_FEATURES = obs.shape[0]  # Flattened observation size
        self.ACTOR_OUTPUT_FEATURES = self.env.action_space.shape[0]* 2  # 2 parameters (alpha, beta) per action dimension

        self.HIDDEN_DIMENSIONS = hidden_dimensions

        self.CRITIC_OUTPUT_FEATURES = 1
        self.DROPOUT = dropout

        self.discount_factor = discount_factor
        self.max_episodes = max_episodes
        self.print_interval = print_interval
        self.PPO_steps=PPO_steps
        self.n_trials=n_trials
        self.epsilon=epsilon
        self.entropy_coefficient=entropy_coefficient
        self.learning_rate=learning_rate

        self.batch_size=batch_size

        # Initialize actor network
        self.actor = BackboneNetwork(
            self.INPUT_FEATURES, self.HIDDEN_DIMENSIONS, self.ACTOR_OUTPUT_FEATURES, self.DROPOUT
        ).to(self.device)

        # Initialize the final layer bias for Beta distribution
        for name, param in self.actor.named_parameters():
            if 'neuralnet.4.bias' in name:  # Adjust index based on your network structure
                # Initialize to produce alpha=beta=2 (uniform-like distribution centered at 0.5)
                param.data.fill_(0.0)  # softplus(0) + 1 = 2
                print(f"Initialized Beta parameters to produce uniform-like distribution")

        # Initialize critic network
        self.critic = BackboneNetwork(
            self.INPUT_FEATURES, self.HIDDEN_DIMENSIONS, self.CRITIC_OUTPUT_FEATURES, self.DROPOUT
        ).to(self.device)

        #Better move the .to(self.device) call separately for both self.actor and self.critic. This ensures the individual parts of the model are moved to the correct device before combined into the actorCritic class
        # Combine into a single actor-critic model
        self.model = actorCritic(self.actor, self.critic)

        try:
            # Try to get the optimizer from torch.optim based on the provided name
            self.optimizer = getattr(torch.optim, optimizer_name)(self.model.parameters(), lr=self.learning_rate)
        except AttributeError:
            # Raise an error if the optimizer_name is not valid
            raise ValueError(f"Optimizer '{optimizer_name}' is not available in torch.optim.")

    def calculate_returns(self, rewards):
        returns = []
        cumulative_reward = 0
        for r in reversed(rewards):
            cumulative_reward = r +cumulative_reward*self.discount_factor
            returns.insert(0, cumulative_reward)
        returns = torch.tensor(returns).to(self.device)

        # Only normalize if we have more than one element to avoid std() warning
        if returns.numel() > 1:
            epsilon = 1e-8  # Small constant to avoid division by zero
            returns_std = returns.std()
            if not torch.isnan(returns_std) and returns_std >= epsilon:
                returns = (returns - returns.mean()) / (returns_std + epsilon)

        #I had conceptual trouble with normalizing the reward by an average, because it seemed to me since we're adding more rewards for earlier timesteps, the cumulative reward for earlier times would be a lot larger. But need to consider dicount facotr.
        # Future rewards contribute significantly to the cumulative return, so earlier timesteps will likely have larger returns.
        #if gamma is close to 0, future rewards have little influence, and the return at each timestep will closely resemble the immediate reward, meaning the pattern might not be as clear.
        return returns

    #The advantage is calculated as the difference between the value predicted by the critic and the expected return from the actions chosen by the actor according to the policy.
    def calculate_advantages(self, returns, values):
        advantages = returns - values

        # Only normalize if we have more than one element to avoid std() warning
        if advantages.numel() > 1:
            epsilon = 1e-8
            advantages_std = advantages.std()
            if not torch.isnan(advantages_std) and advantages_std >= epsilon:
                advantages = (advantages - advantages.mean()) / (advantages_std + epsilon)

        return advantages

    #The standard policy gradient loss is calculated as the product of the policy action probabilities and the advantage function
    #The standard policy gradietn loss cannot make corrections for abrupt policy changes. The surrogate loss modifies the standard loss to restrict the amount the policy can change in each iteration.
    #The surrogate loss is the minimum of (policy ratio X advantage function) and (clipped value of policy ratio X advantage function) where the policy ratio is between the action probabilities according to the old versus new policies and clipping restricts the value to a region near 1.

    def calculate_surrogate_loss(self, actions_log_probability_old, actions_log_probability_new, advantages):
        advantages = advantages.detach()
        # creates a new tensor that shares the same underlying data as the original tensor but breaks the computation graph. This means:
        # The new tensor is treated as a constant with no gradients.
        # Any operations involving this tensor do not affect the gradients of earlier computations in the graph.

        #If the advantages are not detached, the backpropagation of the loss computed using the surrogate_loss would affect both the actor and the critic networks
        # The surrogate loss is meant to update only the policy (actor).
        # Allowing gradients to flow back through the advantages would inadvertently update the critic, potentially disrupting its learning process.

        policy_ratio  = (actions_log_probability_new - actions_log_probability_old).exp()
        surrogate_loss_1 = policy_ratio*advantages
        surrogate_loss_2 = torch.clamp(policy_ratio, min =1.0-self.epsilon, max = 1.0+self.epsilon)*advantages
        surrogate_loss=torch.min(surrogate_loss_1, surrogate_loss_2)
        return surrogate_loss

    #TRAINING THE AGENT
    #Policy loss is the sum of the surrogate loss and the entropy bonus. It is used to update the actor (policy network)
    #Value loss is based on the difference between the value predicted by the critic and the returns (cumulative reward) generated by the policy. This loss is used to update the critic (value network) to make predictions more accurate.

    def calculate_losses(self, surrogate_loss, entropy, returns, value_pred):
        entropy_bonus = self.entropy_coefficient*entropy
        policy_loss = -(surrogate_loss+entropy_bonus).sum()
        value_loss = torch.nn.functional.smooth_l1_loss(returns, value_pred).sum() #helps to smoothen the loss function and makes it less sensitive to outliers.
        return policy_loss, value_loss

    def init_training(self):
        #create a set of buffers as empty arrays. To be used during training to store information
        states = []
        actions = []
        actions_log_probability = []
        values = []
        rewards = []
        done = False
        episode_reward = 0
        return states, actions, actions_log_probability, values, rewards, done, episode_reward

    def forward_pass(self):#this is just the training function (might just want to rename it)
        # # === DETAILED OBJECT ANALYSIS ===
        # import psutil
        # import gc

        # if not hasattr(self, '_episode_counter'):
        #     self._episode_counter = 0
        # self._episode_counter += 1

        # mem_mb = psutil.Process().memory_info().rss / 1024 / 1024

        # # Get ALL objects with "Network" in their type name
        # network_objects = [obj for obj in gc.get_objects() if 'network' in str(type(obj)).lower()]

        # print(f"\n=== EPISODE {self._episode_counter} OBJECT ANALYSIS ===")
        # print(f"Memory: {mem_mb:.1f}MB")
        # print(f"Total objects with 'network' in type: {len(network_objects)}")

        # # Count by exact type
        # type_counts = {}
        # for obj in network_objects:
        #     obj_type = str(type(obj))
        #     type_counts[obj_type] = type_counts.get(obj_type, 0) + 1

        # # Print breakdown
        # for obj_type, count in type_counts.items():
        #     print(f"  {obj_type}: {count}")

        # # Show actual PyPSA Network objects specifically
        # actual_networks = [obj for obj in gc.get_objects() if type(obj).__name__ == 'Network' and 'pypsa' in str(type(obj))]
        # print(f"Actual PyPSA Network objects: {len(actual_networks)}")

        # if len(actual_networks) <= 5:  # Only print if reasonable number
        #     for i, net in enumerate(actual_networks):
        #         print(f"  Network {i+1}: {id(net)} - {type(net)}")

        # network_id = id(self.env.network) if hasattr(self.env, 'network') else None
        # print(f"Current env.network ID: {network_id}")
        # print("=" * 50)
        # # === END ANALYSIS ===

        # Reset environment with seed
        if self.seed is not None:
            state, _ = self.env.reset(seed=self.seed)
        else:
            state, _ = self.env.reset()

        states, actions, actions_log_probability, values, rewards, done, episode_reward = self.init_training()

        # Add this line to track violations
        total_violations = 0

        # # Create fresh network for each episode to avoid memory corruption
        # fresh_network = create_pypsa_network()
        # self.env.network = fresh_network

        state, _ = self.env.reset()  # Gymnasium format returns (obs, info)

        self.model.train() # Set model to training mode

        while True:
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            states.append(state_tensor)

            # Get action predictions and values
            action_mean, value_pred = self.model(state_tensor)



            # Split actor output into alpha and beta parameters
            action_dim = self.env.action_space.shape[0]
            alpha_raw, beta_raw = torch.split(action_mean, action_dim, dim=-1)

            # Ensure alpha, beta > 1 for well-behaved Beta distribution
            alpha = torch.nn.functional.softplus(alpha_raw) + 1.0
            beta = torch.nn.functional.softplus(beta_raw) + 1.0

            # Create Beta distribution for continuous actions in [0,1]
            dist = torch.distributions.Beta(alpha, beta)
            action = dist.sample()

            # No clamping needed - Beta distribution naturally outputs [0,1]
            action_clamped = action

            log_prob_action = dist.log_prob(action).sum(dim=-1)  # Sum over action dimensions

            # Step environment with numpy action
            action_np = action_clamped.detach().cpu().numpy().flatten()
            state, reward, terminated, truncated, info = self.env.step(action_np)
            done = terminated or truncated

            #accumulate violations for the epsiode
            total_violations += sum(info['constraint_violations'].values())

            actions.append(action_clamped)
            actions_log_probability.append(log_prob_action)
            values.append(value_pred)
            rewards.append(reward)
            episode_reward += reward

            if done:
                break

        states=torch.cat(states).to(self.device)#converts the list of individual states into a sinlem tensor that is necessary for later processing
        #Creates a single tensor with dimensions like (N, state_dim), where: N is the number of states collected in the episode; state_dim is the dimensionality of each state.
        #torch.cat() expects a sequence (e.g. list or tuple) of PyTorch tensors as input.
        actions=torch.cat(actions).to(self.device)
        #Note that, in the loop, both state and action are PyTorch tensors so that states and actions are both lists of PyTorch tensors
        actions_log_probability=torch.cat(actions_log_probability).to(self.device)
        values=torch.cat(values).squeeze(-1).to(self.device)# .squeeze removes a dimension of size 1 only from tensor at the specified position, in this case, -1, the last dimesion in the tensor. Note that .squeeze() does not do anything if the size of the dimension at the specified potision is not 1.
        # print(f"rewards NaNs: {torch.isnan(torch.tensor(rewards, dtype=torch.float32)).any()}")
        # print(f"values NaNs: {torch.isnan(torch.tensor(values, dtype=torch.float32)).any()}")
        returns = self.calculate_returns(rewards)
        advantages = self.calculate_advantages(returns, values)

        # print(f"Returns NaNs: {torch.isnan(returns).any()}")
        # print(f"advantages NaNs (after calculation): {torch.isnan(advantages).any()}")

        return episode_reward, states, actions, actions_log_probability, advantages, returns, total_violations


    def update_policy(self,
            states,
            actions,
            actions_log_probability_old,
            advantages,
            returns):
        #print(f"Returns NaNs: {torch.isnan(returns).any()}")
        total_policy_loss = 0
        total_value_loss = 0
        actions_log_probability_old = actions_log_probability_old.detach()
        actions=actions.detach()

        # print(f"Returns NaNs: {torch.isnan(returns).any()}")
        # print(f"advantages NaNs (after calculation): {torch.isnan(advantages).any()}")


        #detach() is used to remove the tensor from the computation graph, meaning no gradients will be calculated for that tensor when performing backpropagation.
        #In this context, it's used to ensure that the old actions and log probabilities do not participate in the gradient computation during the optimization of the policy, as we want to update the model based on the current policy rather than the old one.
        #print(type(states), type(actions),type(actions_log_probability_old), type(advantages), type(returns))
        training_results_dataset= TensorDataset(
                states,
                actions,
                actions_log_probability_old,
                advantages,
                returns) #TensorDataset class expects all the arguments passed to it to be tensors (or other compatible types like NumPy arrays, which will be automatically converted to tensor
        batch_dataset = DataLoader(
                training_results_dataset,
                batch_size=self.batch_size,
                shuffle=False)
        #creates a DataLoader instance in PyTorch, which is used to load the training_results_dataset in batches during training.
        #batch_size defines how many samples will be included in each batch. The dataset will be divided into batches of size BATCH_SIZE. The model will then process one batch at a time, rather than all of the data at once,
        #shuffle argument controls whether or not the data will be shuffled before being split into batches.
        #Because shuffle is false, dataloader will provide the batches in the order the data appears in training_results_dataset. In this case, the batches will be formed from consecutive entries in the dataset, and the observations will appear in the same sequence as they are stored in the dataset.
        for _ in range(self.PPO_steps):
            for batch_idx, (states,actions,actions_log_probability_old, advantages, returns) in enumerate(batch_dataset):
                #get new log prob of actions for all input states
                action_mean, value_pred = self.model(states)
                value_pred = value_pred.squeeze(-1)

                # For continuous actions with Beta distribution
                action_dim = self.env.action_space.shape[0]
                alpha_raw, beta_raw = torch.split(action_mean, action_dim, dim=-1)

                # Ensure alpha, beta > 1 for well-behaved Beta distribution
                alpha = torch.nn.functional.softplus(alpha_raw) + 1.0
                beta = torch.nn.functional.softplus(beta_raw) + 1.0

                probability_distribution_new = torch.distributions.Beta(alpha, beta)
                entropy = probability_distribution_new.entropy().sum(dim=-1)

                #estimate new log probabilities using old actions
                actions_log_probability_new = probability_distribution_new.log_prob(actions).sum(dim=-1)
                # # Check for NaN or Inf in log probabilities
                # if torch.isnan(actions_log_probability_old).any() or torch.isinf(actions_log_probability_old).any():
                #     print("NaN or Inf detected in actions_log_probability_old!")
                #     return  # You can return or handle this case as needed

                # if torch.isnan(actions_log_probability_new).any() or torch.isinf(actions_log_probability_new).any():
                #     print("NaN or Inf detected in actions_log_probability_new!")
                #     return  # You can return or handle this case as needed

                # print(f"actions_log_probability_old NaNs: {torch.isnan(actions_log_probability_old).any()}")
                # print(f"actions_log_probability_new NaNs: {torch.isnan(actions_log_probability_new).any()}")
                # print(f"advantages NaNs: {torch.isnan(advantages).any()}")

                surrogate_loss = self.calculate_surrogate_loss(
                    actions_log_probability_old,
                    actions_log_probability_new,
                    advantages
                )

                # print(f"Surrogate Loss NaNs: {torch.isnan(surrogate_loss).any()}")
                # print(f"Entropy NaNs: {torch.isnan(entropy).any()}")
                # print(f"Returns NaNs: {torch.isnan(returns).any()}")
                # print(f"Value Predictions NaNs: {torch.isnan(value_pred).any()}")

                policy_loss, value_loss = self.calculate_losses(
                    surrogate_loss,
                    entropy,
                    returns,
                    value_pred
                )
                self.optimizer.zero_grad() #clear existing gradietns in the optimizer (so that these don't propagate accross multiple .backward(). Ensures each optimization step uses only the gradients computed during the current batch.

                # Skip backward pass if loss is NaN
                if torch.isnan(policy_loss).any():
                    print("NaN detected in policy_loss - skipping backward pass!")
                    continue
                if torch.isnan(value_loss).any():
                    print("NaN detected in value_loss - skipping backward pass!")
                    continue

                policy_loss.backward() #computes gradients for policy_loss with respect to the agent's parameters
                # #Check for NaN gradients after policy_loss backward
                # for param in self.model.parameters():
                #     if param.grad is not None:  # Check if gradients exist for this parameter
                #         if torch.isnan(param.grad).any():
                #             print("NaN gradient detected in policy_loss!")
                # #             return
                value_loss.backward()
                # Check for NaN gradients after value_loss backwardor param in self.model.parameters():
                # for param in self.model.parameters():
                #     if param.grad is not None:  # Check if gradients exist for this parameter
                #         if torch.isnan(param.grad).any():
                #             print("NaN gradient detected in value_loss!")
                #             return

                self.optimizer.step()
                #The update step is based on the learning rate and other hyperparameters of the optimizer
                # The parameters of the agent are adjusted to reduce the policy and value losses.
                total_policy_loss += policy_loss.item() #accumulate the scalar value of the policy loss for logging/ analysis
                #policy_loss.item() extracts the numerical value of the loss tensor (detaching it from the computational graph).
                #This value is added to total_policy_loss to compute the cumulative loss over all batches in the current PPO step.
                #Result: tracks the total policy loss for the current training epoch
                # The loss over the whole dataset is the sum of the losses over all batches.
                #The training dataset is split into batches during the training process. Each batch represents a subset of the collected training data from one episode.
                # Loss calculation is performed for each batch (policy loss and value loss)
                # for each batch, gradients are calculated with respect to the total loss for that batch and the optimizer then updates the network parameters using these gradients.
                # this is because the surrogate loss is only calculated over a single batch of data
                #look at the formula for surrogate loss.
                # It is written in terms of an expectation ˆ Et[. . .] that indicates the empirical average over a finite batch of samples.
                # This means you have collected a set of data (time steps) from the environment, and you're averaging over these data points. The hat symbol implies you're approximating the true expectation with a finite sample of data from the environment. This empirical average can be computed as the mean of values from the sampled transitions
                # the expectation is taken over all the data you've collected
                #If you're training with multiple batches (i.e., collecting data in chunks), then you can think of the expectation as being computed over each batch.
                #The overall expectation can indeed be seen as the sum of expectations computed for each batch, but The expectation of the sum is generally not exactly equal to the sum of the expectations unless the samples are independent, but in practical reinforcement learning algorithms, it's typically a good enough approximation
                #For samples to be independent, the outcome of one sample must not provide any information about the outcome of another. Specifically, in the context of reinforcement learning, this means that the states, actions, rewards, and subsequent states observed in different time steps or different episodes should be independent of each other.
                total_value_loss += value_loss.item()
                #Notice that we are calculating an empirical average, which is already an approximation on the true value (the true expectation would be the average over an infinite amount of data, and the empirical average is the average over the finite amount of data that we have collected).
                #But furthermore, we are approximating even the empirical average istelf. The empirical average is the average over all our collected datal, but here we actually batch our data, calculate average over each batch and then sum these averages, which is not exaclty equal to the average of the sums (but is a decent approximation).
        return total_policy_loss / self.PPO_steps, total_value_loss / self.PPO_steps

    def train(self):
        train_rewards = []
        # test_rewards = []
        # policy_losses = []
        # value_losses = []
        #lens = []

        for episode in range(1, self.max_episodes + 1):
            # Perform a forward pass and collect experience
            train_reward, states, actions, actions_log_probability, advantages, returns, violations = self.forward_pass()

            # Update the policy using the experience collected
            policy_loss, value_loss = self.update_policy(
                states,
                actions,
                actions_log_probability,
                advantages,
                returns)
            # test_reward = self.evaluate()

            # # Visualize the environment if it supports rendering (currently this is done once each episode - might want to change to once every multiple of episodes)
            # if hasattr(self.env, "render") and callable(getattr(self.env, "render", None)):
            #   self.env.render()

            # Log the results
            # policy_losses.append(policy_loss)
            # value_losses.append(value_loss)
            train_rewards.append(train_reward)
            # # run these when back online
            # self.run["policy_loss"].log(policy_loss)
            # self.run["value_loss"].log(value_loss)
            self.run["train_reward"].log(train_reward)
            self.run["total_violation"].log(violations)

            # Calculate the mean of recent rewards and losses for display
            mean_train_rewards = np.mean(train_rewards[-self.n_trials:])
            #mean_test_rewards = np.mean(test_rewards[-self.n_trials:])
            # mean_abs_policy_loss = np.mean(np.abs(policy_losses[-self.n_trials:]))
            # mean_abs_value_loss = np.mean(np.abs(value_losses[-self.n_trials:]))

            # Print results at specified intervals
            if episode % self.print_interval == 0:
                print(f'Episode: {episode:3} | \
                    Train Rewards: {train_reward:3.1f} \
                    Violations: {violations}\
                    Mean Train Rewards: {mean_train_rewards:3.1f}' )
                    # \
                    # | Mean Abs Policy Loss: {mean_abs_policy_loss:2.2f} \
                    # | Mean Abs Value Loss: {mean_abs_value_loss:2.2f} ')



                                    # | Mean Test Rewards: {mean_test_rewards:3.1f} \
                                    #| "Episode Len: {np.mean(lens[-self.n_trials:])}



            # # Check if reward threshold is reached
            # if mean_test_rewards >= self.reward_threshold:
            #     print(f'Reached reward threshold in {episode} episodes')
            #     break
        # Check if the environment has a close method before calling it
        # if hasattr(self.env, "close") and callable(getattr(self.env, "close", None)):
        #   self.env.close() #Close environment visualisation after training is done.
        return train_rewards

def plot_train_rewards(train_rewards, reward_threshold):
    plt.figure(figsize=(12, 8))
    plt.plot(train_rewards, label='Training Reward')
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Training Reward', fontsize=20)
    plt.hlines(reward_threshold, 0, len(train_rewards), color='y')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

def plot_test_rewards(test_rewards, reward_threshold):
    plt.figure(figsize=(12, 8))
    plt.plot(test_rewards, label='Testing Reward')
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Testing Reward', fontsize=20)
    plt.hlines(reward_threshold, 0, len(test_rewards), color='y')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

def plot_losses(policy_losses, value_losses):
    plt.figure(figsize=(12, 8))
    plt.plot(value_losses, label='Value Losses')
    plt.plot(policy_losses, label='Policy Losses')
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Loss', fontsize=20)
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

In [ ]:
class Env2Gen1LoadConstrReplacement(Env2Gen1LoadConstr):
    """
    Environment using the Replacement reward method instead of Summation.

    Inherits from Env2Gen1LoadConstr but modifies the reward calculation
    to implement the replacement method from the RL-OPF paper.
    """

    def __init__(self,network_file, episode_length=None, constraint_penalty_factor=100, offset_k=2500, input_dir="var_constraint_map"):
        super().__init__(network_file, episode_length=episode_length, constraint_penalty_factor=constraint_penalty_factor, input_dir=input_dir)
        self.offset_k = offset_k

    def calculate_constrained_reward(self):
        """
        Calculate reward using replacement method with pre-calculated offset k.

        Replacement method:
        - If all constraints satisfied: return -J(s) + k
        - If constraints violated: return -P(s)

        Returns:
        --------
        tuple: (reward, constraint_results)
        """
        try:
            # Get base reward from objective function (negative for minimization)
            base_reward = self._calculate_reward()

            # Evaluate constraints
            constraint_results = self._evaluate_all_constraints()

            # Apply replacement method
            if constraint_results['all_satisfied']:
                # All constraints satisfied: return optimization reward + offset k
                constrained_reward = base_reward + self.offset_k
            else:
                # Constraints violated: return only penalty (negative)
                total_violation = float(constraint_results['total_violation'])
                constrained_reward = -self.penalty_factor * total_violation

            # Ensure reward is a scalar
            if hasattr(constrained_reward, '__len__'):
                constrained_reward = float(constrained_reward)

            return constrained_reward, constraint_results

        except Exception as e:
            print(f"Error calculating replacement reward: {e}")
            # Fall back to base reward on error
            return self._calculate_reward(), {
                'all_satisfied': True,
                'violations': {},
                'total_violation': 0.0
            }

    def get_reward_method_info(self):
        """
        Get information about the reward method being used.

        Returns:
        --------
        dict: Information about the reward method
        """
        return {
            'method': 'replacement',
            'offset_k': self.offset_k,
            'k_method': self.k_method,
            'k_samples': self.k_samples,
            'penalty_factor': self.penalty_factor
        }

In [ ]:
# Add this to your main function or create a new sweep script

import time
from datetime import datetime

def run_sweep_replacement():
    # Base parameters (non-swept parameters)
    base_params = {
        "optimizer_name": "Adam",
        "MAX_EPISODES": 100,
        "PRINT_INTERVAL": 10,  # Print less frequently during sweep
        "N_TRIALS": 2,
        "DROPOUT": 0,
        "network_file": "elec_s_5_ec_lc1.0_3h.nc",
        "input_dir": "var_constraint_map"
    }

    # Parameters to sweep (key parameters that affect learning)
    sweep_params = {
        "LEARNING_RATE": [1e-4],
        "EPSILON": [0.1],
        "ENTROPY_COEFFICIENT": [0.01],
        "HIDDEN_DIMENSIONS": [32],
        "PPO_STEPS": [2],
        "BATCH_SIZE": [64],
        "DISCOUNT_FACTOR": [0.99],
        "constraint_penalty_factor":[0,100],
        "episode_length": [1000],
        "env_class": ["Env2Gen1LoadConstr", "Env2Gen1LoadConstrReplacement"] #Will need to change this if you change the env class names
    }

    replacement_env=Env2Gen1LoadConstrReplacement(network_file=base_params["network_file"], input_dir=base_params["input_dir"]) #this is used to calculate the offset k for the replacement reward method

    # Seeds to use for each configuration
    #seeds = [42, 123, 7]  # Using 3 seeds for statistical significance
    seeds = [42]
    #seeds= [123, 7]

    # Generate a subset of combinations to keep the sweep manageable
    # We'll use a more focused approach rather than a full grid search

    # Add some random combinations to explore the space more broadly
    config_seed=42
    import random
    random.seed(config_seed)  # For reproducible random configs

    #after setting the seed, perform sampling to determine k and store the result

    num_random_configs = 1  # Adjust based on how many total runs you want
    random_configs = []

    for _ in range(num_random_configs):
        config = {param: random.choice(values) for param, values in sweep_params.items()}
        random_configs.append(config)

    # Combine priority and random configs
    all_configs = random_configs

    # Print sweep summary
    print(f"Running sweep with {len(all_configs)} configurations and {len(seeds)} seeds")
    print(f"Total runs: {len(all_configs) * len(seeds)}")

    # Run all configurations
    for seed in seeds:
        # Set all random seeds
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)

        #after setting the seed, perform sampling to determine k and store the result
        replacement_reward_offset=calculate_offset_k_initialization(envClass=replacement_env)
        for config_idx, config in enumerate(all_configs):
              # Create a unique run ID
            run_id = f"sweep_{datetime.now().strftime('%Y%m%d')}_{config_idx}_{seed}"

            my_api="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZDg3ZjNlYi04MWI3LTQ1ODctOGIxNS1iNTY3ZjgzMGYzMzYifQ=="

            # Initialize Neptune run
            run = neptune.init_run(
                project='EnergyGridRL/elec-s-5-ec-lc10-3h-GenDispatch',
                api_token=my_api,
                name=f"Sweep-Config{config_idx}-Seed{seed}",
                tags=["hyperparameter_sweep"]
            )

            # Combine base params with this config
            params = {**base_params, **config}
            params["SEED"] = seed

            # Log all parameters
            for key, value in params.items():
                run[f"parameters/{key}"] = value

            run["env_class"]=params["env_class"]
            print(f"\n{'='*50}")
            print(f"Starting run {config_idx+1}/{len(all_configs)}, seed {seed}")
            print(f"Parameters: {params}")
            print(f"{'='*50}\n")

            var_id_to_name, constraints = load_mappings(network_file=params["network_file"], input_dir="var_constraint_map")

            try:

                # Create environment and agent
                if params["env_class"]=="Env2Gen1LoadConstr":
                    env = Env2Gen1LoadConstr(network_file=params["network_file"], episode_length=params["episode_length"], constraint_penalty_factor=params["constraint_penalty_factor"],input_dir=base_params["input_dir"])
                elif params["env_class"]=="Env2Gen1LoadConstrReplacement":
                    env = Env2Gen1LoadConstrReplacement(network_file=params["network_file"], episode_length=params["episode_length"], constraint_penalty_factor=params["constraint_penalty_factor"], offset_k=replacement_reward_offset, input_dir=base_params["input_dir"])
                env.seed(seed)

                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

                agent = PPO_agent(
                    env=env,
                    run=run,
                    device=device,
                    hidden_dimensions=params["HIDDEN_DIMENSIONS"],
                    dropout=params["DROPOUT"],
                    discount_factor=params["DISCOUNT_FACTOR"],
                    optimizer_name=params["optimizer_name"],
                    max_episodes=params["MAX_EPISODES"],
                    print_interval=params["PRINT_INTERVAL"],
                    PPO_steps=params["PPO_STEPS"],
                    n_trials=params["N_TRIALS"],
                    epsilon=params["EPSILON"],
                    entropy_coefficient=params["ENTROPY_COEFFICIENT"],
                    learning_rate=params["LEARNING_RATE"],
                    batch_size=params["BATCH_SIZE"],
                    seed=seed
                )

                run["replacement_reward"].log(replacement_reward_offset)

                # Train the agent
                train_rewards = agent.train()

                # Log final performance metrics
                run["results/final_reward"] = train_rewards[-1]
                run["results/mean_last_100_reward"] = np.mean(train_rewards[-100:])
                run["results/best_reward"] = np.max(train_rewards)
                run["results/baseline_reward"]= evaluate_baseline_reward(network_file=params["network_file"], env=env, var_id_to_name=var_id_to_name)

            except Exception as e:
                print(f"Error in run: {e}")
                run["results/error"] = str(e)

            # Close the Neptune run
            run.stop()

            # Small delay to avoid API rate limits
            time.sleep(1)

In [13]:
# # Add this to your main function or create a new sweep script

# import time
# from datetime import datetime

# def run_sweep_replacement():
#     # Base parameters (non-swept parameters)
#     base_params = {
#         "optimizer_name": "Adam",
#         "MAX_EPISODES": 1000,
#         "PRINT_INTERVAL": 10,  # Print less frequently during sweep
#         "N_TRIALS": 100,
#         "DROPOUT": 0,
#         "network_file": "elec_s_5_ec_lcopt_3h.nc",
#     }

#     # Parameters to sweep (key parameters that affect learning)
#     sweep_params = {
#         "LEARNING_RATE": [1e-4, 3e-4, 1e-3, 3e-3],
#         "EPSILON": [0.1, 0.2, 0.3],
#         "ENTROPY_COEFFICIENT": [0.01, 0.05, 0.1],
#         "HIDDEN_DIMENSIONS": [32, 64, 128],
#         "PPO_STEPS": [8, 16],
#         "BATCH_SIZE": [128, 256],
#         "DISCOUNT_FACTOR": [0.95, 0.99],
#         "constraint_penalty_factor":[0,25,50,100],
#         "episode_length": [2,4,6],
#         "env_class": ["Env2Gen1LoadConstr", "Env2Gen1LoadConstrReplacement"] #Will need to change this if you change the env class names
#     }

#     replacement_env=Env2Gen1LoadConstrReplacement() #this is used to calculate the offset k for the replacement reward method

#     # Seeds to use for each configuration
#     #seeds = [42, 123, 7]  # Using 3 seeds for statistical significance
#     #seeds = [42]
#     seeds= [123, 7]

#     # Generate a subset of combinations to keep the sweep manageable
#     # We'll use a more focused approach rather than a full grid search

#     # Priority configurations based on most promising parameter values
#     # Priority configurations based on most promising parameter values
#     priority_configs = [
#         # Config 1: Higher learning rate, exploration focus
#         {"LEARNING_RATE": 1e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":0, "episode_length": 4},

#         # Config 2: Medium learning rate, balanced approach
#         {"LEARNING_RATE": 3e-4, "EPSILON": 0.2, "ENTROPY_COEFFICIENT": 0.05,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":0,"episode_length": 4},

#         # Config 3: Conservative updates, larger network
#         {"LEARNING_RATE": 1e-4, "EPSILON": 0.1, "ENTROPY_COEFFICIENT": 0.01,
#          "HIDDEN_DIMENSIONS": 128, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":0,"episode_length": 4},

#         # Config 4: Aggressive learning, smaller network
#         {"LEARNING_RATE": 3e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 32, "PPO_STEPS": 8, "BATCH_SIZE": 128,
#          "DISCOUNT_FACTOR": 0.95, "constraint_penalty_factor":0,"episode_length": 4},

#         # Config 1: Higher learning rate, exploration focus
#         {"LEARNING_RATE": 1e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":100,"episode_length": 4},

#         # Config 2: Medium learning rate, balanced approach
#         {"LEARNING_RATE": 3e-4, "EPSILON": 0.2, "ENTROPY_COEFFICIENT": 0.05,
#          "HIDDEN_DIMENSIONS": 64, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":100,"episode_length": 4},

#         # Config 3: Conservative updates, larger network
#         {"LEARNING_RATE": 1e-4, "EPSILON": 0.1, "ENTROPY_COEFFICIENT": 0.01,
#          "HIDDEN_DIMENSIONS": 128, "PPO_STEPS": 16, "BATCH_SIZE": 256,
#          "DISCOUNT_FACTOR": 0.99, "constraint_penalty_factor":100,"episode_length": 4},

#         # Config 4: Aggressive learning, smaller network
#         {"LEARNING_RATE": 3e-3, "EPSILON": 0.3, "ENTROPY_COEFFICIENT": 0.1,
#          "HIDDEN_DIMENSIONS": 32, "PPO_STEPS": 8, "BATCH_SIZE": 128,
#          "DISCOUNT_FACTOR": 0.95, "constraint_penalty_factor":100,"episode_length": 4},
#     ]

#     # Add some random combinations to explore the space more broadly
#     config_seed=42
#     import random
#     random.seed(config_seed)  # For reproducible random configs

#     #after setting the seed, perform sampling to determine k and store the result

#     num_random_configs = 8  # Adjust based on how many total runs you want
#     random_configs = []

#     for _ in range(num_random_configs):
#         config = {param: random.choice(values) for param, values in sweep_params.items()}
#         random_configs.append(config)

#     # Combine priority and random configs
#     all_configs = priority_configs + random_configs

#     # Print sweep summary
#     print(f"Running sweep with {len(all_configs)} configurations and {len(seeds)} seeds")
#     print(f"Total runs: {len(all_configs) * len(seeds)}")

#     # Run all configurations
#     for seed in seeds:
#         # Set all random seeds
#         random.seed(seed)
#         np.random.seed(seed)
#         torch.manual_seed(seed)
#         if torch.cuda.is_available():
#             torch.cuda.manual_seed(seed)
#             torch.cuda.manual_seed_all(seed)

#         #after setting the seed, perform sampling to determine k and store the result
#         replacement_reward_offset=calculate_offset_k_initialization(envClass=replacement_env)
#         for config_idx, config in enumerate(all_configs):
#               # Create a unique run ID
#             run_id = f"sweep_{datetime.now().strftime('%Y%m%d')}_{config_idx}_{seed}"

#             my_api="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZDg3ZjNlYi04MWI3LTQ1ODctOGIxNS1iNTY3ZjgzMGYzMzYifQ=="

#             # Initialize Neptune run
#             run = neptune.init_run(
#                 project='EnergyGridRL/PPO-2snapshots-replacement',
#                 api_token=my_api,
#                 name=f"Sweep-Config{config_idx}-Seed{seed}",
#                 tags=["hyperparameter_sweep"]
#             )

#             # Combine base params with this config
#             params = {**base_params, **config}
#             params["SEED"] = seed

#             # Log all parameters
#             for key, value in params.items():
#                 run[f"parameters/{key}"] = value

#             run["env_class"]=params["env_class"]
#             print(f"\n{'='*50}")
#             print(f"Starting run {config_idx+1}/{len(all_configs)}, seed {seed}")
#             print(f"Parameters: {params}")
#             print(f"{'='*50}\n")

#             try:

#                 # Create environment and agent
#                 if params["env_class"]=="Env2Gen1LoadConstr":
#                     env = Env2Gen1LoadConstr(network_file=params["network_file"], episode_length=params["episode_length"], constraint_penalty_factor=params["constraint_penalty_factor"])
#                 elif params["env_class"]=="Env2Gen1LoadConstrReplacement":
#                     env = Env2Gen1LoadConstrReplacement(network_file=params["network_file"], episode_length=params["episode_length"], constraint_penalty_factor=params["constraint_penalty_factor"], offset_k=replacement_reward_offset)
#                 else:
#                     raise ValueError(f"Invalid environment class: {params["env_class"]}")
#                 env.seed(seed)

#                 device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#                 agent = PPO_agent(
#                     env=env,
#                     run=run,
#                     device=device,
#                     hidden_dimensions=params["HIDDEN_DIMENSIONS"],
#                     dropout=params["DROPOUT"],
#                     discount_factor=params["DISCOUNT_FACTOR"],
#                     optimizer_name=params["optimizer_name"],
#                     max_episodes=params["MAX_EPISODES"],
#                     print_interval=params["PRINT_INTERVAL"],
#                     PPO_steps=params["PPO_STEPS"],
#                     n_trials=params["N_TRIALS"],
#                     epsilon=params["EPSILON"],
#                     entropy_coefficient=params["ENTROPY_COEFFICIENT"],
#                     learning_rate=params["LEARNING_RATE"],
#                     batch_size=params["BATCH_SIZE"],
#                     seed=seed
#                 )

#                 run["replacement_reward"].log(replacement_reward_offset)

#                 # Train the agent
#                 train_rewards = agent.train()

#                 # Log final performance metrics
#                 run["results/final_reward"] = train_rewards[-1]
#                 run["results/mean_last_100_reward"] = np.mean(train_rewards[-100:])
#                 run["results/best_reward"] = np.max(train_rewards)
#                 run["results/baseline_reward"]= evaluate_baseline_reward(network_file=params["network_file"], env=env, agent=agent)

#             except Exception as e:
#                 print(f"Error in run: {e}")
#                 run["results/error"] = str(e)

#             # Close the Neptune run
#             run.stop()

#             # Small delay to avoid API rate limits
#             time.sleep(1)

In [20]:
run_sweep_replacement()

Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA1 0', 'ZA2 0', 'ZA3 0'], dtype='object', name='Bus')
Writing continuous variables.: 100%|██████████| 6/6 [00:00<00:00, 21.45it/s]
Status: warning
Termination condition: infeasible
Solution: 0 primals, 0 duals
Objective: nan
Solver model: available
Solver message: Infeasible

Index(['0'], dtype='object', name='SubNetwork')
Index(['0', '1', '2', '3', '4', '5', '6'], dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA1 0', 'ZA2 0', 'ZA3 0'], dtype='object', name='Bus')


UnboundLocalError: cannot access local variable 'min_load' where it is not associated with a value